In [156]:
class Row:
    def __init__(self, line):
        # The line below was a nasty bug! I missed the starting of a row if the
        # first point was a wall. I only managed to find the bug after I compared
        # the out put of # https://www.reddit.com/r/adventofcode/comments/zsnghh/2022_day_22_part_1_solution_works_only_for_the/
        # with my solution.

        #self.x0 = line.index('.') + 1 BUG
        wall_pos = line.index('#') if '#' in line else 1000_000
        self.x0 = min(line.index('.'), wall_pos) + 1
        self.x1 = len(line)
        self.line = line

    def is_valid_column(self, x):
        return self.x0 <= x <= self.x1

    def is_wall(self, x):
        return self.line[x-1] == "#"

    def right(self, x):
        xx = x + 1
        if xx > self.x1:
            xx = self.x0
        if self.is_wall(xx):
            return None
        else:
            return xx

    def left(self, x):
        xx = x - 1
        if xx < self.x0:
            xx = self.x1
        if self.is_wall(xx):
            return None
        else:
            return xx


class Board:
    def __init__(self, lines):
        self.rows = [Row(line.rstrip()) for line in lines]
        self.y0 = 1
        self.y1 = len(self.rows)

    def row(self, y):
        return self.rows[y-1]

    def start(self):
        row = self.row(self.y0)
        return (row.x0, self.y0)

    def forward(self, pos, orientation):
        x, y = pos
        if orientation == 0:
            return self.right(x, y)
        elif orientation == 1:
            return self.down(x, y)
        elif orientation == 2:
            return self.left(x, y)
        elif orientation == 3:
            return self.up(x, y)
        else:
            raise Exception("Wrong Orientation: %d" % orientation)
        
    def right(self, x, y):
        xx = self.row(y).right(x)
        if xx != None:
            return (xx, y)
        else:
            return None

    def left(self, x, y):
        xx = self.row(y).left(x)
        if xx != None:
            return (xx, y)
        else:
            return None

    def down(self, x, y):
        yy = y + 1
        if yy > self.y1:
            yy = self.y0
        row = self.row(yy)
        if not row.is_valid_column(x):
            # find upmost valid row for column x
            for yy in range(self.y0, self.y1+1):
                row = self.row(yy)
                if row.is_valid_column(x):
                    break
        if row.is_wall(x):
            return None
        else:
            return (x, yy)

    def up(self, x, y):
        yy = y - 1
        if yy < self.y0:
            yy = self.y1
        row = self.row(yy)
        if not row.is_valid_column(x):
            # find lowest valid row for column x
            for yy in range(self.y1, self.y0-1, -1):
                row = self.row(yy)
                if row.is_valid_column(x):
                    break
        if row.is_wall(x):
            return None
        else:
            return (x, yy)

Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^).

In [7]:
def move(board, pos, orientation, num_steps):
    for n in range(num_steps):
        fwd = board.forward(pos, orientation)
        if fwd is None:
            break
        else:
            pos = fwd
    return pos

### With the provided input

**That's not nice!**

In the exmaple we only have alternating turns: R,L,R,L,...  but
in the provided input we have turns like R,R,L,...  !

In [158]:
def read_input2(file_name):
    with open(file_name) as f:
        lines = [x.rstrip() for x in f.readlines()]
    board = Board(lines[:-2])
    instructions = lines[-1]

    res = []
    curr = ""
    for c in instructions:
        if c == "R":
            res.append(int(curr))        
            res.append("R")
            curr = ""
        elif c == "L":
            res.append(int(curr))        
            res.append("L")
            curr = ""
        else:
            curr += c
    res.append(int(curr))

    return board, res

In [159]:
board, moves = read_input2("example")

In [160]:
right_turn = {0: 1, 1: 2, 2: 3, 3: 0}
left_turn = {0: 3, 1: 0, 2: 1, 3: 2}

s = board.start()
orientation = 0
print(s)
for step in moves:
    if type(step) == int:
        s = move(board, s, orientation, step)
    else:
        if step == "R":
            orientation = right_turn[orientation]
        else:
            orientation = left_turn[orientation]

s

(9, 1)


(8, 6)

With the provided input

In [161]:
board, moves = read_input2("input")
s = board.start()
orientation = 0
print(s)
for step in moves:
    if type(step) == int:
        s = move(board, s, orientation, step)
    else:
        if step == "R":
            orientation = right_turn[orientation]
        else:
            orientation = left_turn[orientation]

(51, 1)


In [162]:
s, orientation

((107, 1), 0)

In [154]:
1000 * 1 + 4 * 107 + 0

1428

In [163]:
marker = {0: '>', 1: 'v', 2: '<', 3: '^'}

def mark_trail(board, pos, orientation):
    x, y = pos
    row = board.row(y)
    cs = [c for c in row.line]
    cs[x-1] = marker[orientation]
    row.line = "".join(cs)

def trail(board, pos, orientation, num_steps):
    mark_trail(board, pos, orientation)
    for n in range(num_steps):
        fwd = board.forward(pos, orientation)
        if fwd is None:
            break
        else:
            pos = fwd
            mark_trail(board, pos, orientation)

    return pos

In [ ]:
board, moves = read_input2("input")
s = board.start()
orientation = 0
def print_board(board):
    for row in board.rows:
        print(row.line)

for step in moves[:20]:
    print(step)
    if type(step) == int:
        s = trail(board, s, orientation, step)
    else:
        if step == "R":
            orientation = right_turn[orientation]
        else:
            orientation = left_turn[orientation]
        mark_trail(board, s, orientation)

    print_board(board)
    print()